This workbook is to create a new dataframe using random informations and calculate balance after each transaction and holding periods

In [1]:
# importing libraries
import random
import numpy as np
import pandas as pd
import datetime

#### Functions to create data (dates, numbers, indicators)

In [2]:
# This is functions to create a random data
# Random dates
def random_dates(obs):
    # Random dates 
    start_date = datetime.date(2019, 1, 1)
    end_date = datetime.date(2020, 2, 1)

    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days

    random_date_list = [datetime.date(2019, 1, 1)]
    for i in range(0,obs): 
        random_number_of_days = random.randrange(days_between_dates)
        random_date = start_date + datetime.timedelta(days=random_number_of_days)
        random_date_list.append(random_date)

#     print(random_date_list)
    return random_date_list

# Random numbers
def random_numbers(obs):
    randomlist = [100]
    for i in range(0,obs):
        n = random.randint(20,40)
        randomlist.append(n)
#     print(randomlist)
    return randomlist

# buy and sell indicators
def buy_sell_numbers(obs):
    buy_sell = [0]
    for i in range(0,obs):
        n = random.randint(0,1)
        buy_sell.append(n)

    for i, num in enumerate(buy_sell):
        if num == 0:
            buy_sell[i] = "BUY"
        else:
            buy_sell[i] = "SELL"

#     print(buy_sell)
    return buy_sell


#### iterate over the lists to create a new dataframe with random numbers

In [3]:
# Randomly generating data 

# how many observations? (n-1)
n = 24
# doing it 4 times to create the dataframe
coin_list = ['BTC', 'ETH', 'ADA', 'DOGE']
# creating a dataframe
df = pd.DataFrame(columns = ["Date", "coin_name", "BUY/SELL", "value"])

for i in range(len(coin_list)):
#     dates
    dates = random_dates(n)
#     random numbers
    numbers = random_numbers(n)
#     buy and sell indicator
    buy_sell = buy_sell_numbers(n)
#     coin name feature
    coin_name = [coin_list[i] for n in range(25)]
    
#     putting all in a dataframe
    df1 = pd.DataFrame(zip(dates, coin_name, buy_sell, numbers), 
                      columns = ["Date", "coin_name", "BUY/SELL", "value"])
    df = df.append(df1)


Now the dataframe has been created. 
Sorting the values by the date, then reset index and creating a new variable to indicate the negative values.

In [4]:
# making sure to sort the values based on dates
df = df.sort_values('Date')
# !!!! IMPORTANT - make sure to reset index
df = df.reset_index(drop=True)
# creating a feature to capture the negative value when selling
df['new'] = df['value']
df.loc[df['BUY/SELL']=='SELL', 'new'] = df.loc[df['BUY/SELL']=='SELL', 'new']*-1
df

,Date,coin_name,BUY/SELL,value,new
0,2019-01-01,BTC,BUY,100,100
1,2019-01-01,DOGE,BUY,100,100
2,2019-01-01,ETH,BUY,100,100
3,2019-01-01,ADA,BUY,100,100
4,2019-01-03,ETH,SELL,35,-35
...,...,...,...,...,...
95,2020-01-19,ADA,BUY,40,40
96,2020-01-22,ETH,BUY,34,34
97,2020-01-24,ETH,SELL,23,-23
98,2020-01-26,BTC,BUY,20,20


#### Calculating the daily balance after each transaction

In [5]:
# code to get a balance for each transaction
df['balance'] = 0
for i in range(len(df)): 
    df.loc[i,'balance'] = df.loc[0:i, 'new'].sum()

In [6]:
df.head()

,Date,coin_name,BUY/SELL,value,new,balance
0,2019-01-01,BTC,BUY,100,100,100
1,2019-01-01,DOGE,BUY,100,100,200
2,2019-01-01,ETH,BUY,100,100,300
3,2019-01-01,ADA,BUY,100,100,400
4,2019-01-03,ETH,SELL,35,-35,365


#### Working out the holding periods

Steps: 
Based on the lists, a new dataframe has been created for each element in the list and then tag the rows when sell and the first buy. 
The consequtive buys are removed to workout the day difference. 
And then merge this data with the main datafile for only sell transaction to workout the holding periods. 

In [7]:
# now workingout the holding period
# sort the data for each coins
df = df.sort_values(by=['coin_name','Date'])

# Here I'm only looking at when they made the sell trades 
# (I didn't incorporated the quantity as I wasn't sure what exactly you want to capture here)

# creating a new dataframe to append the outputs
df1 = pd.DataFrame()
coin_list = ['BTC', 'ETH', 'ADA', 'DOGE']

# iterate through each coins in the dataframe
for name in coin_list:
#     new data for selected coin
    df_n = df.groupby(['coin_name']).get_group(name)
#     !!!! IMPORTANT - make sure to reset index
    df_n = df_n.reset_index(drop=True)
    
#     identifying when the transaction is a sell
    df_n['tag_sell'] = 0
    df_n.loc[df_n['BUY/SELL']=='SELL', 'tag_sell'] = 1

#     identifying when the transaction is a buy (only the first occurence)
    df_n['tag_buy'] = 0
    df_n.loc[0,'tag_buy'] = 1
    df_n.loc[(df_n['BUY/SELL']=='BUY') & (df_n['BUY/SELL'].shift(1)=='SELL'), 'tag_buy'] = 1

#     keeping only those identified (consequetive BUY transactions are removed)
    df_n = df_n.loc[(df_n['tag_sell']==1) | (df_n['tag_buy']==1)]

#     Working out the holding period in days
    df_n['holding_days'] = df_n['Date'] - df_n['Date'].shift(1)
    
#     making sure that we only keep sell transactions
    df_n = df_n.loc[(df_n['BUY/SELL']=='SELL')]

#     dropping features we don't need
    df_n.drop(['tag_sell','tag_buy'], axis=1, inplace=True)
    
#     appending each coins to a dataframe
    df1 = df1.append(df_n)

In [8]:
# merge holding periods
df_new = df.merge(df1[['Date','coin_name','BUY/SELL','value','holding_days']], 
                  how='left', on=['Date','coin_name','BUY/SELL','value'])


### Now we have balance after each transaction and the holding periods (in days) when transaction is a sell. 

In [9]:
# now you can see there is balance and holding in days
df_new.head(50)

,Date,coin_name,BUY/SELL,value,new,balance,holding_days
0,2019-01-01,ADA,BUY,100,100,400,NaT
1,2019-01-31,ADA,BUY,27,27,376,NaT
2,2019-03-05,ADA,BUY,31,31,284,NaT
3,2019-04-15,ADA,SELL,24,-24,243,104 days
4,2019-05-02,ADA,SELL,35,-35,243,17 days
5,2019-05-21,ADA,BUY,32,32,304,NaT
6,2019-07-03,ADA,SELL,36,-36,459,43 days
7,2019-07-07,ADA,SELL,20,-20,439,4 days
8,2019-07-14,ADA,BUY,27,27,500,NaT
9,2019-07-19,ADA,SELL,23,-23,477,5 days
